In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("insurance.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df[['age', 'affordibility']], df.bought_insurance, test_size=0.2, random_state=25)


In [ ]:
len(x_train)

22

In [ ]:
x_train_scaled = x_train.copy()
x_train_scaled['age'] = x_train_scaled['age']/100

x_test_scaled = x_test.copy()
x_test_scaled['age'] = x_test_scaled['age']/100

In [ ]:
x_train_scaled

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1
19,0.18,1
25,0.54,1
16,0.25,0
20,0.21,1
3,0.52,0


In [ ]:
model = keras.Sequential(
    [
        keras.layers.Dense(1, input_shape=(2,), activation="sigmoid", name="layer1", kernel_initializer='ones', bias_initializer='zeros'),
    ]
)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x_train_scaled, y_train, epochs=5000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 7ms/step - loss: 0.5321 - accuracy: 0.7273
Epoch 2502/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.5321 - accuracy: 0.7273
Epoch 2503/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.5321 - accuracy: 0.7273
Epoch 2504/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.5320 - accuracy: 0.7273
Epoch 2505/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.5320 - accuracy: 0.7273
Epoch 2506/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.5320 - accuracy: 0.7273
Epoch 2507/5000
1/1 [==============================] - 0s 13ms/step - loss: 0.5319 - accuracy: 0.7273
Epoch 2508/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.5319 - accuracy: 0.7273
Epoch 2509/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.5318 - accuracy: 0.7273
Epoch 2510/5000
1/1 [==============================] -

In [ ]:
model.predict(x_test_scaled)

array([[0.70548487],
       [0.3556955 ],
       [0.16827849],
       [0.47801173],
       [0.7260697 ],
       [0.8294984 ]], dtype=float32)

In [ ]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [ ]:
coff, intercept = model.get_weights()
coff, intercept

(array([[5.060867 ],
        [1.4086502]], dtype=float32), array([-2.9137027], dtype=float32))

# Building it from scratch:

In [ ]:
def sigmoid(x):
  import math
  return 1 / (1 + math.exp(-x))
sigmoid(7)

0.9990889488055994

In [ ]:
def prediction_function(age, affordibility):
  weighted_sum = coff[0]*age + coff[1]*affordibility + intercept
  return sigmoid(weighted_sum)

In [ ]:
prediction_function(0.47, 1)

0.7054848693136117

In [ ]:
prediction_function(0.18, 0)

0.11891484999991211

In [ ]:
def log_loss(y_true, y_predicted):
  epsilon = 1e-15
  y_predicted_new = [max(i, epsilon) for i in y_predicted]
  y_predicted_new = [min(i, 1-epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [ ]:
def sigmoid_numpy(x):
  return 1/(1+np.exp(-x))

sigmoid_numpy(np.array([12, 0, 1]))

array([0.99999386, 0.5       , 0.73105858])

In [ ]:
def gradient_descent(age, affordability, y_true, epochs):
  # w1, w2, bias
  w1 = w2 = 1
  bias = 0
  rate = 0.5 
  n = len(age)

  for i in range(epochs):
    weighted_sum = w1 * age + w2 * affordability + bias
    y_predicted = sigmoid_numpy(weighted_sum)

    loss = log_loss(y_true, y_predicted)

    w1d = (1/n)*np.dot(np.transpose(age), (y_predicted-y_true))
    w2d = (1/n)*np.dot(np.transpose(affordability), (y_predicted-y_true))
    bias_d = np.mean(y_predicted - y_true)

    w1 = w1 - rate * w1d
    w2 = w2 - rate * w2d
    bias = bias - rate * bias_d

    print(f'Epoch: {i}, w1:{w1}, w2:{w2}, bias:{bias}, loss: {loss}')

  return w1, w2, bias




In [ ]:
gradient_descent(x_train_scaled['age'], x_train_scaled['affordibility'], y_train, 1000)

Epoch: 0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss: 0.7113403233723417
Epoch: 1, w1:0.9556229728273669, w2:0.9058873696677865, bias:-0.2122349122718517, loss: 0.6812647787377568
Epoch: 2, w1:0.9416488476693794, w2:0.8719790823960313, bias:-0.29775789977965383, loss: 0.6591474252715025
Epoch: 3, w1:0.9323916996249162, w2:0.8457541517722915, bias:-0.37150947240035115, loss: 0.6431523291301916
Epoch: 4, w1:0.9272267472726993, w2:0.8262362885332687, bias:-0.4350664302689159, loss: 0.6316873063379158
Epoch: 5, w1:0.9255469396815343, w2:0.8124402814952774, bias:-0.4899449005893882, loss: 0.6234717079975919
Epoch: 6, w1:0.9267936114129968, w2:0.8034375029757677, bias:-0.5375299543522855, loss: 0.6175321183044205
Epoch: 7, w1:0.93047170420295, w2:0.7983920007454487, bias:-0.5790424270894964, loss: 0.6131591858705934
Epoch: 8, w1:0.9361540784567943, w2:0.7965748796787705, bias:-0.6155315088627656, loss: 0.6098518179750948
Epoch: 9, w1:0.9434791243557358, w2:0.

(8.27023951605617, 1.6332228396255535, -4.425953608965704)